In [8]:
# Import standard python libraries
import numpy as np
from numpy import pi
from numpy.fft import fft, rfft
import math, json

# Import standard Qiskit libraries
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, IBMQ, transpile, execute
from qiskit.result import marginal_counts
import qiskit.providers.aer.noise as noise
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error

# Load IBM Quantum account
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-12-20 21:51:29,763: Credentials are already in use. The existing account in the session will be replaced.


In [ ]:
# Hybrid quantum gap estimation
# Real-device simulation
# Mid-circuit measurement
# Control input qubit orientation

# Set up parameters
g = round(0.2, 1)
Nc = 2
eta = 0.3
dw = 0.25*eta

if (math.ceil(14.0/dw) % 2) == 0:
    L = math.ceil(14.0/dw)
else:
    L = math.ceil(14.0/dw) + 1

M = 16
shots = 1024
rep_delay = 0.0005 # range: [0,0.0005], default: 0.00025

# Set up input qubit
s = []
for j in range (Nc):
    s.append('0')
str1 = ""
InQ = str1.join(s)
    
# Loop for input qubit orientation
for i0 in range (0,26):
    
    if i0 == 25:
        imax = 1
    else:
        imax = 2
    
    # Set up circuits
    qreg = QuantumRegister(Nc, 'q')
    creg = ClassicalRegister(Nc*L*imax, 'c')
    qc = QuantumCircuit(qreg, creg)
        
    # Loop for input qubit orientation
    for i in range (imax):

        # Set up input qubit orientation
        theta0 = round(0.02 * i0 + 0.01 * i, 2)
        theta = pi * theta0
    
        # Loop for time series
        for j in range (L):

            # Set up time angle
            phi = j*2*pi/L/dw
        
            # reset qubits
            qc.reset(qreg)
        
            # Rotation of input qubits
            for n in range (Nc):
                qc.ry(theta, qreg[n])
    
            # Loop for Trotter steps
            for m in range (M):

                # Sequence of gates for time evolution
                for n in range (Nc):
                    qc.rx(-2*phi/M, qreg[n])
                
                for n in range (Nc-1):
                    qc.rzz(-2*g*phi/M, qreg[n], qreg[n+1])
                    
            # Rotation of measurement basis
            for n in range (Nc):
                qc.ry(-theta, qreg[n])
        
            # Measure qubits
            assign_qubits = [Nc*L*i + Nc*j + k for k in range(Nc)]
            qc.measure(qreg, assign_qubits)
    
    print("running...")
    
    # Construct the unnitary for time evolution
    backend = provider.get_backend("ibmq_manila")
    qc_trans = transpile(qc, backend, optimization_level = 1, initial_layout = [k for k in range(Nc)])
    result = execute(qc_trans, backend, shots = shots, rep_delay = rep_delay).result()
    
    # Loop for input qubit orientation
    for i in range (imax):

        # Set up input qubit orientation
        theta0 = round(0.02 * i0 + 0.01 * i, 2)
        theta = pi * theta0

        # Print start message
        print("theta/pi =", theta0,"started")
    
        # Allocate arrays
        propagator = []
    
        # Loop for time series
        for j in range (L):
    
            # Set up time angle
            phi = j*2*pi/L/dw
        
            # Marginal measurement counts without mitigation
            assign_qubits = [Nc*L*i + Nc*j + k for k in range(Nc)]
            counts = marginal_counts(result, indices = assign_qubits).get_counts()
            
            if InQ in counts:
                propagator.append(2*pi/L/dw*np.exp(-eta*phi)*counts[InQ]/shots)
            else:
                propagator.append(0)
    
            print(j+1, end="\r")
    
        # Flatten array
        propagator = np.ravel(propagator)

        # Evaluate the spectral function via Fourier transform
        spectral_function_0 = np.real(np.fft.ifft(propagator))*L/pi

        # Rearrange array
        spectral_function = np.append(spectral_function_0[int(L/2):L], spectral_function_0[0:int(L/2)+1])
    
        # Define lists
        frequency = [(j-L/2)*dw for j in range(L+1)]
        spectral_function = [spectral_function[j] for j in range(L+1)]

        # Save data
        with open('data_spectral_N%s_g%sp%s_eta%sp%s_M%s_theta%sp%s_realdevice_ibmq_manila_11242022' % 
                (int(Nc),int(g),str(g).split(".")[-1],int(eta),str(eta).split(".")[-1],int(M),int(theta0),str(theta0).split(".")[-1]), 'w') as f:
            f.write(json.dumps(spectral_function))
    
        # Print end message
        print("done")